In [14]:
# Dependencies
import pandas as pd

In [15]:
# Load & read in geocodes file
data_load = "../RawData/all-geocodes-v2019.csv"
raw_data = pd.read_csv(data_load, header=4,  encoding = 'unicode_escape')
raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,'000,0,0,0,United States
1,40,1,'000,0,0,0,Alabama
2,50,1,'001,0,0,0,Autauga County
3,50,1,'003,0,0,0,Baldwin County
4,50,1,'005,0,0,0,Barbour County


In [16]:
# drop unwanted columns
raw_data = pd.DataFrame(raw_data, columns=['Summary Level', 'State Code (FIPS)', 'County Code (FIPS)', 'Area Name (including legal/statistical area description)'])

raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,'000,United States
1,40,1,'000,Alabama
2,50,1,'001,Autauga County
3,50,1,'003,Baldwin County
4,50,1,'005,Barbour County


In [17]:
# slicing off ' from county code
raw_data['County Code (FIPS)'] = raw_data['County Code (FIPS)'].str[1:]

raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description)
0,10,0,000,United States
1,40,1,000,Alabama
2,50,1,001,Autauga County
3,50,1,003,Baldwin County
4,50,1,005,Barbour County


In [18]:
# Concatinate state and county code
raw_data['combined fips'] = raw_data['State Code (FIPS)'].map(str) + raw_data['County Code (FIPS)']
raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description),combined fips
0,10,0,000,United States,0000
1,40,1,000,Alabama,1000
2,50,1,001,Autauga County,1001
3,50,1,003,Baldwin County,1003
4,50,1,005,Barbour County,1005


In [19]:
# Create state table - filter for states
states = raw_data.loc[raw_data['Summary Level'] == 40]

# Select colums wanted in correct order
state = pd.DataFrame(states, columns=['combined fips', 'Area Name (including legal/statistical area description)'])

# Rename columns
state = state.rename(columns={'combined fips':'state_fips', 'Area Name (including legal/statistical area description)': 'state_name'}).set_index('state_fips')

# Export state table to csv
state.to_csv('state.csv')

In [20]:
# Create county table - filter for county
county = raw_data.loc[raw_data['Summary Level'] == 50]

# Select colums wanted in correct order
county = pd.DataFrame(county, columns=['combined fips', 'Area Name (including legal/statistical area description)'])

# Rename columns
county = county.rename(columns={'combined fips':'county_fips', 'Area Name (including legal/statistical area description)': 'county_name'}).set_index('county_fips')

# Export state table to csv
county.to_csv('county.csv')

In [21]:
# Create a state_fip in raw data
raw_data['state_fips'] = raw_data['State Code (FIPS)'].map(str) + '000'
raw_data.head()

,Summary Level,State Code (FIPS),County Code (FIPS),Area Name (including legal/statistical area description),combined fips,state_fips
0,10,0,000,United States,0000,0000
1,40,1,000,Alabama,1000,1000
2,50,1,001,Autauga County,1001,1000
3,50,1,003,Baldwin County,1003,1000
4,50,1,005,Barbour County,1005,1000


In [22]:
# filter on 50 to create county_state table
county_state_fips = raw_data.loc[raw_data['Summary Level'] == 50]
county_state = pd.DataFrame(county_state_fips, columns=['combined fips', 'state_fips'])

# rename and index columns
county_state = county_state.rename(columns={'combined fips':'county_fips'}).set_index('county_fips')

# save county state to csv
county_state.to_csv('county_state.csv')

# Dataframe to postgres

In [23]:
# import dependencies
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
from passwords import password

## County table

In [24]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "county"

try:

    frame = county.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'county' already exists.


## State table

In [25]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "state"

try:

    frame = state.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'state' already exists.


## County_state table

In [26]:
# Create an engine instance
alchemyEngine   = create_engine('postgresql+psycopg2://postgres:' + password + '@cwru-bootcamp6.cbgqvzvry5u3.us-east-2.rds.amazonaws.com/postgres', pool_recycle=3600)

# Connect to PostgreSQL server
postgreSQLConnection = alchemyEngine.connect()
postgreSQLTable = "county_state"

try:

    frame = county_state.to_sql(postgreSQLTable, postgreSQLConnection, if_exists='fail')

except ValueError as vx:

    print(vx)

except Exception as ex:  

    print(ex)

else:

    print("PostgreSQL Table %s has been created successfully."%postgreSQLTable)

finally:

    postgreSQLConnection.close()

Table 'county_state' already exists.
